## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-11-49-20 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
1,2025-07-12-11-44-37 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...
2,2025-07-12-11-23-19 +0000,bbc,"Labour not defending workers, says Unite after...",https://www.bbc.com/news/articles/cn819l4229mo
3,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo
4,2025-07-12-10-00-09 +0000,nyt,A Beloved Canadian Chocolate Bar Disappears Fr...,https://www.nytimes.com/2025/07/12/world/canad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,13
0,air,5
1,india,5
158,canada,4
271,year,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-07-12-11-44-37 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...,26
45,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,26
48,2025-07-11-19-04-37 +0000,nyt,Trump Says NATO Countries Will Buy Weapons to ...,https://www.nytimes.com/2025/07/11/world/europ...,25
0,2025-07-12-11-49-20 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...,24
60,2025-07-11-15-51-14 +0000,nyt,Rubio and China’s Top Envoy Vie for Influence ...,https://www.nytimes.com/2025/07/11/us/politics...,24


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,26,2025-07-12-11-44-37 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...
0,24,2025-07-12-11-49-20 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
67,17,2025-07-11-14-39-16 +0000,bbc,Ex-MP faces two-year wait for betting offences...,https://www.bbc.com/news/articles/c056v0m56pyo
16,15,2025-07-12-01-12-57 +0000,bbc,'They were just kids': Mother mourns sons kill...,https://www.bbc.com/news/articles/cy9xgrrq54go
58,14,2025-07-11-16-24-19 +0000,bbc,Russians tell the BBC how they think the Ukrai...,https://www.bbc.com/news/videos/c1wp55wqq0lo
36,14,2025-07-11-21-06-00 +0000,wsj,Court Quashes Sept. 11 Plea Bargains That Mili...,https://www.wsj.com/us-news/law/september-11-p...
62,13,2025-07-11-15-41-42 +0000,bbc,Gaza's largest functioning hospital facing dis...,https://www.bbc.com/news/articles/cdx5zeywgrgo
73,13,2025-07-11-13-07-16 +0000,nyt,"As Climate Change Heats Up Europe’s Summers, A...",https://www.nytimes.com/2025/07/11/theater/avi...
45,11,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
60,11,2025-07-11-15-51-14 +0000,nyt,Rubio and China’s Top Envoy Vie for Influence ...,https://www.nytimes.com/2025/07/11/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
